##Importing necessary libraries

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

Mounting Google Drive

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


##Data Formatting - Inputting just slash separated haikus for our Char RNN

In [ ]:
# importing library
import pandas as pd
  
# Then loading csv file
df = pd.read_csv('/content/drive/MyDrive/CIS530-Project/Data/haikus.csv')

In [ ]:
df

,0,1,2,source,0_syllables,1_syllables,2_syllables
0,Memorial Day --,a shadow for each,white cross,tempslibres,5,5,2
1,spring rain -,as the doctor speaks,i think of lilacs,tempslibres,"2,3",5,5
2,spring moonset --,a rice ball for,breakfast,tempslibres,"3,4",4,2
3,sunny afternoon,an old man lingers,near the mailbox,tempslibres,5,5,4
4,cinco de mayo,horses roll,in the shallows,tempslibres,5,3,4
...,...,...,...,...,...,...,...
143132,I'm not asking did,you say it nor clarify,what you said neither,twaiku,5,7,5
143133,You are truly a,moron or a liar I'm,inclined to think both,twaiku,5,7,5
143134,Ain't no selfie on,this earth that's gonna make me,like Theresa May,twaiku,5,7,5
143135,is doing a great,job turning Independents,into Democrats,twaiku,5,7,5


In [ ]:
cols = ['0','1','2']
newcol = ['/'.join(i) for i in df[cols].astype(str).values]
df = df.assign(haiku=newcol).drop(cols, 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
print(df['haiku'][0])

Memorial Day --/a shadow for each/white cross$


In [ ]:
df

,source,0_syllables,1_syllables,2_syllables,haiku
0,tempslibres,5,5,2,Memorial Day --/a shadow for each/white cross
1,tempslibres,"2,3",5,5,spring rain -/as the doctor speaks/i think of ...
2,tempslibres,"3,4",4,2,spring moonset --/a rice ball for/breakfast
3,tempslibres,5,5,4,sunny afternoon/an old man lingers/near the ma...
4,tempslibres,5,3,4,cinco de mayo/horses roll/in the shallows
...,...,...,...,...,...
143132,twaiku,5,7,5,I'm not asking did/ you say it nor clarify/wha...
143133,twaiku,5,7,5,You are truly a/ moron or a liar I'm/inclined ...
143134,twaiku,5,7,5,Ain't no selfie on/ this earth that's gonna ma...
143135,twaiku,5,7,5,is doing a great/ job turning Independents/int...


In [ ]:
df['haiku'] = df['haiku'] + '$'

In [ ]:
df['haiku'].to_csv(r'/content/just_haikus.txt', header=None, index=None, sep='\n', mode='a')

Combining our two datasets into /$ format

In [ ]:
data = data2 = ""
  
# Reading data from file1
with open('/content/drive/MyDrive/CIS530-Project/Data/just_haikus.txt') as fp:
    data = fp.read()
  
# Reading data from file2
with open('/content/drive/MyDrive/CIS530-Project/Data/lines.txt') as fp:
    data2 = fp.read()
  
# Merging 2 files
# To add the data of file2
# from next line
data += "\n"
data += data2
  
with open ('/content/drive/MyDrive/CIS530-Project/Data/combined_just_haikus.txt', 'w') as fp:
    fp.write(data)

## CHAR RNN  - main implementation starts from here

In [ ]:
with open('/content/drive/MyDrive/CIS530-Project/Data/combined_just_haikus.txt', 'r') as f:
    text = f.read()

In [ ]:
text[0:200]

'Memorial Day --/a shadow for each/white cross$\nspring rain -/as the doctor speaks/i think of lilacs$\nspring moonset --/a rice ball for/breakfast$\nsunny afternoon/an old man lingers/near the mailbox$\nc'

In [ ]:
# encode the text and map each character to an integer and vice versa
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
# check that the function works as expected
test_seq = np.array([[0, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)
print(one_hot)

[[[1. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [ ]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        #lstm layer - here number of characters is the embedding size as the input is one hot encoded
        self.lstm=nn.LSTM(len(self.chars),n_hidden,n_layers,
                          dropout=drop_prob,batch_first=True)
        
        #dropout layer
        self.dropout=nn.Dropout(drop_prob)
        
        #output layer
        self.fc=nn.Linear(n_hidden,len(self.chars))
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
# define and print the net
n_hidden = 512
n_layers = 2
net = CharRNN(chars, n_hidden, n_layers)
print(net)
batch_size = 128
seq_length = 100
n_epochs =  30
# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

CharRNN(
  (lstm): LSTM(117, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=117, bias=True)
)
Epoch: 1/30... Step: 10... Loss: 3.3365... Val Loss: 3.1073
Epoch: 1/30... Step: 20... Loss: 3.2583... Val Loss: 3.0633
Epoch: 1/30... Step: 30... Loss: 3.2214... Val Loss: 3.0335
Epoch: 1/30... Step: 40... Loss: 3.2389... Val Loss: 3.0469
Epoch: 1/30... Step: 50... Loss: 3.1966... Val Loss: 3.0409
Epoch: 1/30... Step: 60... Loss: 3.1918... Val Loss: 3.0351
Epoch: 1/30... Step: 70... Loss: 3.1912... Val Loss: 3.0389
Epoch: 1/30... Step: 80... Loss: 3.1650... Val Loss: 3.0346
Epoch: 1/30... Step: 90... Loss: 3.1890... Val Loss: 3.0324
Epoch: 1/30... Step: 100... Loss: 3.1854... Val Loss: 3.0297
Epoch: 1/30... Step: 110... Loss: 3.1476... Val Loss: 3.0094
Epoch: 1/30... Step: 120... Loss: 3.1327... Val Loss: 2.9843
Epoch: 1/30... Step: 130... Loss: 3.0531... Val Loss: 2.9450
Epoch: 1/30... Step: 140... Los

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)
        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)
    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    return ''.join(chars)

In [ ]:
# change the name, for saving multiple files
model_name = 'poem_4_epoch.net'
checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}
with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

Implementation ends here

##OUTPUT PROCESSING FOR GRUEN

In [ ]:
#primes used - nature, shadow, beauty, family, autumn, a, e, i, o, u

In [ ]:
output = sample(net, 20000, prime='nature', top_k=2)

In [ ]:
print(output)

nature/are the street the courteous/words to see the sand$
in the summer star/the shape of the complaining/of the sun in the car$
summer rain/the shadows of the companions are/some strangers are stars$
the stranger stands through a star/a star of an old man was an anticipate/and so much they will be there$
then to their street too the same thing in the shore/and the street are the beauty to see the stars on the window/a star is to seek to think the stars of a street$
a street and the strain of this war that i was so tired and some somethen the same work/is an answer to this soul i have seen the state of all/the wind is something that is that i have said that they're so so funny and strength$
and it is a beautiful stars/and the wind was the stream of the strangers/and start to the sound of the window on the shore$
a strange of the straight of the stars of the shadows/of a straight of the colors of a colour of the word/and the streets on the shore of a stars are so soft$
the world's star

In [ ]:
haikus = output.split('$\n')

In [ ]:
haiku_syllable_wise = [];
for h in haikus:
  temp_h = h.split('/')
  haiku_syllable_wise.append(temp_h)

  

In [ ]:
# df = pd.DataFrame(haiku_syllable_wise)
df = pd.DataFrame(haiku_syllable_wise, columns =['sent_1', 'sent_2','sent_3']) 


In [ ]:
df

,sent_1,sent_2,sent_3
0,uble,shadow is the best friend,and it was the same
1,I wanna get out,of my life and I'm so sad,I want a shitty
2,I have a sendence,and I want the best for them,to be there to be
3,I hate when people,see the same then staying at,me and stay too much
4,I won't be something,to be a star and I was,starting to see you
...,...,...,...
293,I'm still a stream to,the past to start the same as,the same time again
294,I haven't sent my,shit together to see you,a start of this shift
295,Today was the best,person to see the story,I won't see this storm
296,This is the best thing,to do in my life I have,a bitch to stay too


In [ ]:
df = df[:-1]

In [ ]:
df.to_csv('/content/haiku_charrnn_output_.csv',columns = ['sent_1','sent_2','sent_3'])

In [ ]:
# csv_files = ['/content/haiku_charrnn_output.csv',
#              '/content/haiku_charrnn_output_2.csv',
#              '/content/haiku_charrnn_output_3.csv',
#              '/content/haiku_charrnn_output_4.csv',
#              '/content/haiku_charrnn_output_5.csv',
#              '/content/haiku_charrnn_output_6.csv',
#              '/content/haiku_charrnn_output_7.csv',
#              '/content/haiku_charrnn_output_8.csv',
#              '/content/haiku_charrnn_output_9.csv',
#              '/content/haiku_charrnn_output_10.csv']

In [ ]:
# df_append = pd.DataFrame()
# #append all files together
# for file in csv_files:
#             df_temp = pd.read_csv(file)
#             df_temp = df_temp[:-1]
#             df_append = df_append.append(df_temp, ignore_index=True)
# df_append

,Unnamed: 0,sent_1,sent_2,sent_3
0,0,flowers was,a straight of a child and shadow,on a cold street stars
1,1,a shadow and the colours,of a complex shaped to see the,stars and sheets and the streets are so sad an...
2,2,and the sound of the straight strange thinks t...,is a stream that i saw,the world to see the stars of the wind
3,3,the sun is a silence the streets of a breeze,in the windows on the stars and street and shaped,to start a barnes of street and the candle shade
4,4,that standing at the sun and the can of strand,and so the summer stars,and stars are still there
...,...,...,...,...
1453,291,I wonder if I,had a star to this season,and I was still soon
1454,292,I wanna see the,same thing that is all and that,I want is the same
1455,293,I'm still a stream to,the past to start the same as,the same time again
1456,294,I haven't sent my,shit together to see you,a start of this shift


In [ ]:
# df_append.to_csv('/content/haiku_charrnn_final_output.csv',columns = ['sent_1','sent_2','sent_3'])